In [27]:
##This script was used for executing IOKR with both DNA and aminoacid kmers
## Sigma = 10000 . This is like gamma X
## GammaY = 0.1
## Lambda = 0.01

In [28]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\dictionariesForKmerCounts

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\dictionariesForKmerCounts


In [29]:
import numpy as np

In [30]:
kmerToColumn={}
i=0
numberOfClusters=0
inputFile = open("length2Processed.txt")
for line in inputFile:
    if line [0:3]== "BGC":
        numberOfClusters +=1
        continue
    line = line.strip("\n")
    line = line.strip(";")
    line = line.split(";")
    for kmer in line:
        kmer=kmer.split(":")
        if kmer[0] not in kmerToColumn:
            kmerToColumn[kmer[0]]=i
            i+=1
inputFile.close()

In [31]:
inputVectorMatrix= np.zeros((numberOfClusters,i))
i=0
inputFile = open("length2Processed.txt")
for line in inputFile:
    if line [0:3]== "BGC":
        continue
    thisClusterDictionary={}
    line = line.strip("\n")
    line = line.strip(";")
    line = line.split(";")
    for kmer in line:
        kmer=kmer.split(":")
        thisClusterDictionary[kmer[0]] =kmer[1]
    for key in kmerToColumn:
        if key in thisClusterDictionary.keys():
            inputVectorMatrix[i,kmerToColumn[key]] = thisClusterDictionary[key]
    i+=1
inputFile.close()

In [32]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\kmersIOKR

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\kmersIOKR


In [33]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String and returns a numpy array
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [34]:
def gaussianKernelForKmers (vector1, vector2):##Changed the formula of the gaussian kernel
    sigma=10000
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-(euclideanDistance**2)/(2*sigma**2))
    return kernelValue

In [35]:
def gaussianKernelForFingerprint (vector1, vector2):##Normal gaussian kernel formula for fingerprints
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [36]:
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [37]:
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [38]:
inputVectorList=[] ##This name is just so we can use the rest of the script without changing the name everywhere (modified from IOKR with pfam counts)
for i in range (0, numberOfClusters):
    inputVectorList.append(inputVectorMatrix[i])

In [39]:
##Sets parameters for the training part of the formula (lambda*Identity*XMatrix) and inverses it
dimensionXMatrix= len (inputVectorList)
lambdaa=0.01 #lambda seems to be a special character in python so lets put a double aa
generalXMatrix = np.zeros((dimensionXMatrix,dimensionXMatrix))
for i in range (0, dimensionXMatrix):
    for j in range (0, dimensionXMatrix):
        generalXMatrix [i,j]=gaussianKernelForKmers(inputVectorList[i],inputVectorList[j])#Kx with upper case
dimensionYMatrix= len (fingerprintVectorList)
generalYMatrix = np.zeros((dimensionYMatrix,dimensionYMatrix))
for i in range (0, dimensionYMatrix):
    for j in range (0, dimensionYMatrix):
        generalYMatrix [i,j]=gaussianKernelForFingerprint(fingerprintVectorList[i],fingerprintVectorList[j])#Ky with upper case

In [40]:
import random
#Creates a list of Random Numbers as big as the number of Clusters we have
#Then we will select which ones we have for each subset
listOfRandomNumbers=random.sample(range(dimensionXMatrix), dimensionXMatrix)
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]

In [41]:
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [42]:
def calculatingIOKRScore (xIndex, yIndex):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=generalXMatrix[iIndex,xIndex]
        ky[i]=generalYMatrix[iIndex,yIndex]
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [43]:
score1 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score1[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score1",score1)

In [44]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training set
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [45]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score2",score2)

In [46]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training set
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [47]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score3",score3)

In [48]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training set
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [49]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score4",score4)

In [50]:
sampleOf206Numbers=listOfRandomNumbers[825:]
sampleOf825Numbers= listOfRandomNumbers[:825]
##Creates the training part of the formula once you know the specific clusters in the training set
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [51]:
score5 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score5",score5)